In [13]:
!pip install nltk


In [14]:
import json
import re
from string import punctuation
from nltk import word_tokenize 
from nltk.util import ngrams
import nltk
nltk.download('punkt')
from nltk.util import pad_sequence
import math


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json

--2021-05-06 18:50:01--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json.3’

dev-v2.0.json.3     100%[===================>]   4.17M  21.8MB/s    in 0.2s    

2021-05-06 18:50:01 (21.8 MB/s) - ‘dev-v2.0.json.3’ saved [4370528/4370528]

--2021-05-06 18:50:01--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json.2’

train-v2.0.json.2 

In [16]:
with open("train-v2.0.json",'r') as load_f:
    load_dict = json.load(load_f)


In [17]:
# load_dict.get("data")


In [ ]:
# load_dict.get("data")[0].get("paragraphs")[0]
load_dict.get("data")[1]

In [21]:
with open("dev-evaluate-v2.0-in1.txt", 'r') as load_f:
    pred_dict = json.load(load_f)
pred_dict

FileNotFoundError: ignored

#Citation:
https://arxiv.org/pdf/1606.05250.pdf section 6

Rajpurkar, P.; Zhang, J.; Lopyrev, K.; and Liang, P. 2016. SQuAD: 100,000+ Questions for Machine Comprehension of Text. In Proceedings of the 2016 Conference on Empiri- cal Methods in Natural Language Processing, 2383–2392.

In [22]:
#sample input: ans_pred: answers for all questions
#ans_true: list of list of all true answers, if is_impossible==True, it is an empty list


def evaluate(ans_pred,ans_true):
    EM_sum=0
    F1_sum=0

    for i in range(len(ans_pred)):
        f1_max=0
        EM_max=0
        if len(ans_true[i])==0:
            f1_max=f1(ans_pred[i],"")
            EM_max=EM(ans_pred[i],"")
        for one_true in ans_true[i]:
            if f1(ans_pred[i],one_true)>f1_max:
                f1_max=f1(ans_pred[i],one_true)
            if EM(ans_pred[i],one_true)>EM_max:
                EM_max=EM(ans_pred[i],one_true)
        F1_sum+=f1_max
        EM_sum+=EM_max
    return EM_sum/len(ans_pred), F1_sum/len(ans_pred)
        
        
        

def f1(ans_pred,ans_true):
    pred_bag = normalize_text(ans_pred).split()
    true_bag = normalize_text(ans_true).split()
    if len(pred_bag) == 0 or len(true_bag) == 0:
        return int(pred_bag == true_bag)

    overlap = set(pred_bag) & set(true_bag)
    
    if len(overlap) == 0:
        return 0
    
    precision = len(overlap) / len(pred_bag)
    recall = len(overlap) / len(true_bag)
    
    return 2 * (precision * recall) / (precision + recall)

def EM(ans_pred,ans_true):
    return int(normalize_text(ans_pred) == normalize_text(ans_true))

    
def normalize_text(text):
    reg = '[^0-9A-Za-z\u4e00-\u9fa5]'
    text=re.sub(reg, ' ', text)
    text=re.sub(r'(^|\s)(a|an|the)($|\s)', ' ', text)
    text=text.lower()
    text=" ".join(text.split())
    return text



# testing

In [23]:
para = load_dict.get("data")[0].get("paragraphs")[0]

answers=[[i.get("text") for i in para.get("qas")[i].get("answers")] for i in range(len(para.get("qas")))]
context=para.get("context")

pred=["France",'10th and 11th centuries','Denmark, Iceland and Norway',"rollo","10th","","","","ss"]
print(answers)
evaluate(pred,answers)


[['in the late 1990s'], ['singing and dancing'], ['2003'], ['Houston, Texas'], ['late 1990s'], ["Destiny's Child"], ['Dangerously in Love'], ['Mathew Knowles'], ['late 1990s'], ['lead singer'], ['Dangerously in Love'], ['2003'], ['five'], ['lead singer'], ['Dangerously in Love']]


(0.0, 0.03174603174603175)

#baseline

https://arxiv.org/pdf/1606.05250.pdf section 5

Rajpurkar, P.; Zhang, J.; Lopyrev, K.; and Liang, P. 2016. SQuAD: 100,000+ Questions for Machine Comprehension of Text. In Proceedings of the 2016 Conference on Empiri- cal Methods in Natural Language Processing, 2383–2392.

In [24]:
# !pip install stanfordcorenlp
# from stanfordcorenlp import StanfordCoreNLP,CoreNLPDependencyParser
# nlp = CoreNLPDependencyParser(url='http://localhost:9000')
!pip install benepar
%tensorflow_version 1.x
import benepar
benepar.download('benepar_en3')
from benepar.spacy_plugin import BeneparComponent
import spacy
nlp = spacy.load('en')
nlp.add_pipe(BeneparComponent('benepar_en3'))


     |████████████████████████████████| 3.3MB 10.2MB/s 
     |████████████████████████████████| 2.1MB 36.2MB/s 
     |████████████████████████████████| 1.2MB 44.1MB/s 
     |████████████████████████████████| 901kB 38.7MB/s 
  Created wheel for benepar: filename=benepar-0.2.0-cp37-none-any.whl size=37647 sha256=197b438a02f5839792f9b4e37ee39c8e5fd883c490aa878fde051572420fcc48
  Stored in directory: /root/.cache/pip/wheels/dd/90/28/513063023646df7774be9401c440a5f13b48bdbab15a67fc42
Successfully built benepar
TensorFlow 1.x selected.
[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Unzipping models/benepar_en3.zip.


/usr/local/lib/python3.7/dist-packages/benepar/spacy_plugin.py:12: FutureWarning: BeneparComponent and NonConstituentException have been moved to the benepar module. Use `from benepar import BeneparComponent, NonConstituentException` instead of benepar.spacy_plugin. The benepar.spacy_plugin namespace is deprecated and will be removed in a future version.
  FutureWarning,


In [25]:
def candidates(paragraph):
  candidate_answers=paragraph.split(".")
  # consituent_parsing=nlp.word_tokenize(paragraph)
  return candidate_answers

def bigram_overlap(answer,question):
  answer="~ "+answer+" ~"
  question="~ "+question+" ~"
  answer_bigram = set(list(ngrams(word_tokenize(answer), 2)) )
  question_bigram = set(list(ngrams(word_tokenize(question), 2)) )
  bigram_overlap=len(answer_bigram.intersection(question_bigram))
  return bigram_overlap

def IC(w,passage):
  C=0
  for word in passage.split():
    C+=(word==w)

  return math.log(1+ 1/C)

def sliding_window(question,answer,passage):
  A=set([i for i in answer])
  Q=set([i for i in question])
  P=passage.split()
  S=A.union(Q)
  maxsw=0
  for j in range(len(P)):
    sw=0
    for w in range(len(S)):
      if j+w<len(P) and P[j+w] in S:
        sw+=IC(P[j+w],passage)
    if sw>maxsw:
      maxsw=sw
  return maxsw

def get_one(candidate_answers,question,passage):
  maxoverlap=0
  best_answers=list()
  for one_answer in candidate_answers:
    # sentence= #get sentence containing answer excluding answer itself
    overlap=bigram_overlap(one_answer,question)
    if overlap>maxoverlap:
      maxoverlap=overlap
      best_answers=[one_answer]
    elif overlap==maxoverlap and overlap ==0:
      continue
    elif overlap==maxoverlap:
      best_answers.append(one_answer)
  bestsw=0
  if len(best_answers)==0:
    return ""
  for answer in best_answers:
    sw=sliding_window(question,answer,passage)
    if sw>bestsw:
      bestsw=sw
      bestanswer=answer
  if bestsw==0:
    return ""
  return bestanswer


def baseline(questions,passage):
  answers=list()
  candidate_answers=candidates(passage)
  for one_question in questions:
    answers.append(get_one(candidate_answers,one_question,passage))
  return answers

In [26]:
def ev_baseline():
  full_answer_list=list()
  full_pred_list=list()
  for para_index in range(len(load_dict.get("data")[0].get("paragraphs"))):
    para = load_dict.get("data")[0].get("paragraphs")[para_index]
    answers=[[i.get("text") for i in para.get("qas")[i].get("answers")] for i in range(len(para.get("qas")))]
    questions=[ para.get("qas")[i].get("question") for i in range(len(para.get("qas")))]
    context=para.get("context")
    base=baseline(questions,context)
    for i in range(len(base)):
      full_pred_list.append(base[i])
      full_answer_list.append(answers[i])
  a=evaluate(full_pred_list,full_answer_list)
  return a
ev_baseline()

(0.0, 0.09104955270490495)

In [28]:
with open("train-v2.0.json",'r') as load_f:
    load_dict = json.load(load_f)
ev_baseline()

(0.0, 0.09104955270490495)

#testing

In [29]:
para = load_dict.get("data")[0].get("paragraphs")[5]
questions=[ para.get("qas")[i].get("question") for i in range(len(para.get("qas")))]

answers=[[i.get("text") for i in para.get("qas")[i].get("answers")] for i in range(len(para.get("qas")))]
context=para.get("context")
base=baseline(questions,context)
# pred=["France",'10th and 11th centuries','Denmark, Iceland and Norway',"rollo","10th","","","","ss"]
# evaluate(base,answers)
print(questions)
print(answers)
print(base)

["Who decided to place Beyonce's group in Star Search the talent show?", 'In 1995, who decided to manage the girls singing group?', 'Who was the first record label to give the girls a record deal?', 'Who brought Beyonce to California and enter her group in Star Search?', "In what year did Beyonce's father quit his job to manage her group?", "What large record company recorded Beyonce's group's first album?", "What record company first signed Beyonce's group and later cut them?", 'At what age did Beyonce meet LaTavia Robertson?', 'How old was Beyoncé when she met LaTavia Roberson?', 'What was the name of the first group Beyoncé was a part of?', "Who placed Girl's Tyme in Star Search?", 'When did Beyoncé begin to manage the girl group?', 'Who signed the girl group on October 5, 1995?']
[['Arne Frager'], ["Beyoncé's father"], ['Elektra Records'], ['Arne Frager'], ['1995'], ['Sony Music'], ['Elektra Records'], ['age eight'], ['eight'], ["Girl's Tyme"], ['Arne Frager'], ['1995'], ["Dwayne W